In [5]:
import json
f = open("./dialogs/q_a.json")

data = json.load(f)
data

{'1707831919.223918': {'date': '2024-02-13',
  'time_utc': '13:45:19',
  'question': 'привет',
  'response': 'Привет!',
  'time_spent': 0.36}}

# генерация ключей

In [25]:
import secrets

# Генерация случайного ключа
def generate_api_key():
    # Генерация случайной строки из символов шестнадцатеричного формата
    api_key = secrets.token_hex(32)  # Вы можете выбрать любую длину ключа, 32 байта (64 символа шестнадцатеричного формата) в данном случае
    return api_key

# Генерация API ключа
api_key = generate_api_key()
api_key

'716beeee49e8981b54619f960579f798f40dba42e1cf88260a77ddee6ae7b94f'

# POST

In [9]:
import requests
import json

URL = "http://localhost:8008/llm"
POST_JSON = {
    "user_id": 1050,
    "question_string": "Чем ты можешь быть мне полезен?"
}
r = requests.post(
    url=URL,
    headers={
        "Content-Type": "application/json",
        "API": "131e12aa46252f4da6920dd2feccc94978688eab3a96337ba4b67a945eac1308"
    },
    json=POST_JSON
)

response = r.json()
response

{'user_id': 1050,
 'date': '2024-02-13',
 'time': '08:59:14',
 'client_ip': '127.0.0.1',
 'user_question': 'Чем ты можешь быть мне полезен?',
 'ai_response': 'Я могу помочь вам с проверкой результатов лотереи, дать консультации по выбору лотереи и ее номиналов, рассказать о специальных предложениях и акциях. Также я могу помочь с вопросами о процессе регистрации и использования сервиса.',
 'response_time': 4.41}

# Загрузка данных

### Загрузка данных из таблицы вопросов-ответов

In [3]:
from lib.vector_db_setup import get_texts, upload_to_vectorstore, vectorstore_query, get_chroma_client
import chromadb
from chromadb.config import Settings
import warnings
warnings.simplefilter("ignore", UserWarning)
import warnings

texts = get_texts(
    file_name="questions_stolot.csv",
    content_column='question'
)

upload_to_vectorstore(
    texts,
    collection_name='book'
)

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

Выбраны данные из файла: ./SOURCE_DOCUMENTS/questions_stolot.csv
Загрузка датасета: SUCCESS
Загрузка модели для эмбеддингов: SUCCESS
Подключение к CHROMADB: SUCCESS
Была обнаружена и удалена существующая коллекция с именем "book"
Создание коллекции book: SUCCESS
Загрузка данных в CHROMADB: SUCCESS
Подключение к CHROMADB: SUCCESS


In [5]:
question = 'правила лотереи'
question = 'когда тираж забавы'
vector_db_response = vectorstore_query(
    collection=collection,
    source_file_type='table',
    question = question,
    n_results=5
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

============ QUESTION: ============
когда тираж забавы
============ VECTOR DB RESPONSE: ============
Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faqТиражи быстрой лотереи Забава от Русского лото проходят каждые 15 минут часа


### Загрузка данных из .txt-файла

In [1]:
from lib.vector_db_setup import get_texts, upload_to_vectorstore, vectorstore_query, get_chroma_client
import chromadb
from chromadb.config import Settings

texts = get_texts(
    file_name="answers_stoloto.txt"
)

upload_to_vectorstore(
    texts,
    collection_name='book'
)

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

Выбраны данные из файла: ./SOURCE_DOCUMENTS/answers_stoloto.txt
Загрузка датасета: SUCCESS


/Users/aweeu/Desktop/wp3_chatbot/9_3_rag_streamlit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Загрузка модели для эмбеддингов: SUCCESS
Подключение к CHROMADB: SUCCESS
Была обнаружена и удалена существующая коллекция с именем "book"
Создание коллекции book: SUCCESS
Загрузка данных в CHROMADB: SUCCESS
Подключение к CHROMADB: SUCCESS


In [2]:
question = 'правила лотереи'
vector_db_response = vectorstore_query(
    collection=collection,
    source_file_type='text',
    question = question,
    n_results=5
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

2024-02-14 14:05:29.531746 [W:onnxruntime:, helper.cc:67 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-02-14 14:05:29.532391 [W:onnxruntime:, coreml_execution_provider.cc:81 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 49 number of nodes in the graph: 323 number of nodes supported by CoreML: 231


============ QUESTION: ============
правила лотереи
============ VECTOR DB RESPONSE: ============
Если вы ввели всё правильно и ваш друг получил оповещения по СМС и имейлу, но не видит подаренного билета в личном кабинете, необходимо сообщить об этом в службу поддержки пользователей (нажмите диалоговую иконку в правом углу любой страницы сайта). Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq
Тиражная комиссия следит за ходом каждого розыгрыша и подтверждает его результаты — подписывает протокол тиражной комиссии и официальную таблицу результатов проведения тиража. тиража (он указан на билете);— тираж на момент проверки уже состоялся (дата тиража указана на билете);— вы правильно выбрали название лотереи (например, если у вас билет «Русского лото», в пункте проверки выберите это название).Если сразу после просмотра трансляции программы «У нас выигрывают!» на на билете);— тираж на момент проверки уже состоялся (дата тиража указана на билете);— вы пр

# llm

In [6]:
from lib.vector_db_setup import get_texts, upload_to_vectorstore, vectorstore_query, get_chroma_client

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

Подключение к CHROMADB: SUCCESS


In [2]:
from peft import PeftConfig
from transformers import AutoTokenizer
from lib.llm_setup import initialize_model, generate_llm_response

# lora_adapter = "IlyaGusev/saiga_mistral_7b"
lora_adapter = "IlyaGusev/saiga2_13b_lora"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

# Настройка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # ???

model = initialize_model(
    base_model=base_model,
    lora_adapter=lora_adapter,
    # bnb=True
)

/root/ml_projects/chatbot/9_3_rag_streamlit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.21s/it]


In [3]:
# watch -n0.2 nvidia-smi
question = 'как играть в забаву'

output_message = generate_llm_response(
    question=question,
    model=model,
    collection=collection,
    tokenizer=tokenizer
)

print(output_message)

Ответ: Для того чтобы сыграть в забаву, необходимо выбрать 5 цифр из диапазона от 1 до 30 и оплатить билет. Билеты можно приобрести у продавца или онлайн на сайте www.stoloto.ru. После этого нужно ждать объявления результатов игры. Если все ваши цифры совпадут с выбранными случайным образом, то вы получите приз.


# langchain

In [2]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import os 
import chromadb
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

/root/ml_projects/chatbot/9_3_rag_streamlit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
MODEL_NAME = "IlyaGusev/saiga2_13b_lora"
config = PeftConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = model.merge_and_unload() #  ОБЯЗАТЕЛЬНО!!!
model.eval()

tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 2.27MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 2.84MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 9.28MB/s]
config.json: 100%|██████████| 610/610 [00:00<00:00, 1.07MB/s]
model.safetensors.index.json: 100%|██████████| 33.4k/33.4k [00:00<00:00, 53.5MB/s]
model-00001-of-00003.safetensors:  10%|█         | 1.03G/9.95G [00:16<02:25, 61.4MB/s]


KeyboardInterrupt: 

In [13]:
from transformers import GenerationConfig
from langchain import PromptTemplate

generation_config = GenerationConfig(
  bos_token_id = 1,
  do_sample = True,
  eos_token_id = 2,
  max_length = 2048,
  repetition_penalty=1.1,
  no_repeat_ngram_size=15,
  pad_token_id = 0,
  temperature = 0.3,
  top_p = 0.9
)

template = "Ты - русскоязычный ассистент Изя. Ты отвечаешь на вопрос пользователя, используя только данную тебе информацию. Не пытайся придумать ответ.\nИнформация: {context}\nВопрос:{question}\nОтвет:"
prompt = PromptTemplate.from_template(template)

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

chroma_client = chromadb.HttpClient(settings=Settings(
    allow_reset=True,
    chroma_api_impl='chromadb.api.fastapi.FastAPI',
    chroma_server_host='localhost',
    chroma_server_http_port='8000')
)
collection = chroma_client.get_collection(name="book", embedding_function=embedding_function)

db = Chroma(client=chroma_client,
            collection_name='book',
            embedding_function=embeddings)

retriever = db.as_retriever(
    search_kwargs={"k":2}
)

In [14]:
question = "как тебя зовут"
response = db.similarity_search_with_score(question)
response

[(Document(page_content='Как играть в забаву', metadata={'answer': 'Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq', 'id': '136', 'url': 'False'}),
  0.18254726069627136),
 (Document(page_content='как играть выиграть в забаву', metadata={'answer': 'Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq', 'id': '138', 'url': 'False'}),
  0.25993116606365574),
 (Document(page_content='Как избавиться от сыгранных билетов в личном кабинете?', metadata={'answer': 'Самостоятельное удаление билетов невозможно — они автоматически перемещаются в архив по истечении 180 дней с момента проведения тиража/розыгрыша.', 'id': '13', 'url': 'False'}),
  0.3138025999069214),
 (Document(page_content='Как купить билет', metadata={'answer': 'Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.', 'id': '38', 'url': 'https

In [15]:
pipe = pipeline(
        'text-generation',
        model = model,
        tokenizer = tokenizer,
        generation_config=generation_config
)

llm = HuggingFacePipeline(
    pipeline=pipe
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [16]:
question = 'как играть в забаву'
generated_text = qa.invoke({"query": question})

print(f"============QUESTION:============\n{generated_text['query']}\n")
print(f"============RESPONSE:============\n{generated_text['result']}")

============QUESTION:============
как играть в забаву

============RESPONSE:============
 Забава - это развлечение или игровое действие, которое не требует особого уровня подготовки и специальных знаний. Для того чтобы начать играть в забаву, вам нужно лишь найти собеседника или группу людей, желающих принять участие в этом развлечении. Забавы могут быть как одиночными, так и коллективными, и их правила могут быть различными.


In [17]:
generated_text

{'query': 'как играть в забаву',
 'result': ' Забава - это развлечение или игровое действие, которое не требует особого уровня подготовки и специальных знаний. Для того чтобы начать играть в забаву, вам нужно лишь найти собеседника или группу людей, желающих принять участие в этом развлечении. Забавы могут быть как одиночными, так и коллективными, и их правила могут быть различными.',
 'source_documents': [Document(page_content='Как играть в забаву', metadata={'answer': 'Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq', 'id': '136', 'url': 'False'}),
  Document(page_content='как играть выиграть в забаву', metadata={'answer': 'Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq', 'id': '138', 'url': 'False'})]}